## Machine leaning project
This project is about wine reviews dataset that was found in Kaggle.

Project team members: Annely Liivas, Velda Lauringson, Eduard Rudi

In [1]:
import pandas as pd
import gensim
from gensim.models import Word2Vec

In [2]:
import nltk
import re

In [29]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rudi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rudi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [30]:
data_loc = 'data/'

In [31]:
data1 = pd.read_csv(data_loc + 'winemag-data_first150k.csv', index_col=0)
data2 = pd.read_csv(data_loc + 'winemag-data-130k-v2.csv', index_col=0)

In [32]:
#Panen andmed kokku
dataset = pd.concat([data1,data2])

In [33]:
# Reset index
dataset.reset_index(drop=True)

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280896,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...
280897,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Pinot Noir,Citation,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon)
280898,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Gewürztraminer,Domaine Gresser,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...
280899,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Pinot Gris,Domaine Marcel Deiss,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace)


In [34]:
#Vaatan, mis toimub
dataset.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN


In [35]:
#Suvaline hulk kirjeldusi (25000)
subset = dataset.description.iloc[:25000]

In [36]:
#Esimene kirjeldus näiteks
subset[0]

'This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.'

In [37]:
reviews_lower = [x.lower() for x in subset]

In [38]:
reviews_lower[0]

'this tremendous 100% varietal wine hails from oakville and was aged over three years in oak. juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. enjoy 2022–2030.'

In [39]:
#Puhastan teksti sodist, aga võib vaielda, et äkki ikkagi on vaja mingit tüüpi arve säilitada (nt aastaarvud)
cleaned_reviews = [re.sub('[^a-zA-Z]', ' ', x) for x in reviews_lower]
cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]

In [40]:
cleaned_reviews[0]

'this tremendous varietal wine hails from oakville and was aged over three years in oak juicy red cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy '

In [41]:
reviews = [nltk.sent_tokenize(x) for x in cleaned_reviews]

In [42]:
reviews = [y for x in reviews for y in x]

In [43]:
reviews[0]

'this tremendous varietal wine hails from oakville and was aged over three years in oak juicy red cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy'

In [44]:
words = [nltk.word_tokenize(x) for x in reviews]

In [45]:
for i in range(len(words)):
    words[i] = [w for w in words[i] if w not in stopwords.words('english')]

In [47]:
#Põhimõtteliselt on korpus valmis
words[0]

['tremendous',
 'varietal',
 'wine',
 'hails',
 'oakville',
 'aged',
 'three',
 'years',
 'oak',
 'juicy',
 'red',
 'cherry',
 'fruit',
 'compelling',
 'hint',
 'caramel',
 'greet',
 'palate',
 'framed',
 'elegant',
 'fine',
 'tannins',
 'subtle',
 'minty',
 'tone',
 'background',
 'balanced',
 'rewarding',
 'start',
 'finish',
 'years',
 'ahead',
 'develop',
 'nuance',
 'enjoy']

In [48]:
# Mudel
model = gensim.models.Word2Vec(words, vector_size = 200 , window = 4 , min_count = 15,
                               seed = 1000, hs = 0, negative = 10,workers=16, epochs = 25)

In [49]:
#Kümme kõige lähedasemat sõna sõnale "magus"
model.wv.most_similar('sweet')

[('syrup', 0.39332252740859985),
 ('preserves', 0.36493930220603943),
 ('sweetness', 0.3646295368671417),
 ('ripe', 0.3537130653858185),
 ('upfront', 0.3505045175552368),
 ('flavors', 0.34368905425071716),
 ('butterscotch', 0.3422863483428955),
 ('pie', 0.3371044993400574),
 ('filled', 0.3346055746078491),
 ('exotic', 0.330181747674942)]

In [50]:
#Kümme kõige lähedasemat sõna sõnale "kuiv"
model.wv.most_similar('dry')

[('dryness', 0.4134575426578522),
 ('semisweet', 0.358644962310791),
 ('refreshingly', 0.33089908957481384),
 ('bite', 0.32968488335609436),
 ('cutting', 0.32748886942863464),
 ('spry', 0.3143460750579834),
 ('drinkable', 0.3141527473926544),
 ('powered', 0.31322696805000305),
 ('lip', 0.3130287826061249),
 ('brisk', 0.3105144202709198)]

Päris asjani on tükk maad veel. Ma jooksutasin tervel datal ka (mingi paar tundi läks) ja tulid palju paremad vektorid, aga siis pani arvuti pange :') Mu arust võiks jõuda selleni, et saaks veinisoovitusi nii karakterit kirjeldavate märksõnade kui veininime järgi (ehk mis veinid oleksid konkreetsele veinile kõige sarnasemad). Word2veci saab ära kasutada selles mõttes, et märksõna järgi otsides ei pea istuma me kinni ühe kindla sõna küljes (nt kui otsime kuiva veini, siis me ei saa vastu ainult neid, mille kirjelduses on konkreetselt sõna "kuiv"), vaid saame võtta arvesse ka selle sünonüüme/semantiliselt lähedasi sõnu. Siinse katsetuse põhjal saame ehk algatada arutelu?